In [3]:
import pandas as pd, numpy as np, os

In [2]:
output_dir = '/mnt/share/costeffectiveness/results/vivarium_ciff_sam/v7.1.0_full_model_vicious_cyle_diarrhea_remission/ciff_sam/2022_03_08_00_15_24/count_data/'

In [5]:
[c for c in os.listdir(output_dir) if '.csv' in c]

['diarrhea_transition_count.csv',
 'wasting_transition_count.csv',
 'deaths.csv',
 'diarrhea_state_person_time.csv',
 'wasting_state_person_time.csv',
 'disease_transition_count.csv',
 'births.csv',
 'population.csv',
 'ylds.csv',
 'ylls.csv',
 'disease_state_person_time.csv',
 'stunting_state_person_time.csv']

# birthweight

In [6]:
births = pd.read_csv(output_dir + 'births.csv').drop(columns='Unnamed: 0')
births.head()

,sex,year,measure,input_draw,scenario,value,insecticide_treated_nets,maternal_supplementation,maternal_malnutrition
0,female,2022,birth_weight_sum,29,baseline,0.000000e+00,covered,bep,cat2
1,female,2022,birth_weight_sum,29,baseline,0.000000e+00,uncovered,bep,cat2
2,female,2022,birth_weight_sum,29,baseline,0.000000e+00,covered,mmn,cat2
3,female,2022,birth_weight_sum,29,baseline,0.000000e+00,uncovered,mmn,cat2
4,female,2022,birth_weight_sum,29,baseline,4.489095e+06,covered,ifa,cat2


In [9]:
birth_numerator = births.loc[(births.measure=='birth_weight_sum')
                            &(births.scenario=='baseline')].groupby(['input_draw']).sum()
birth_denominator = births.loc[(births.measure=='total_births')
                            &(births.scenario=='baseline')].groupby(['input_draw']).sum()

(birth_numerator / birth_denominator).describe(percentiles=[0.025,0.975])

,year,value
count,11.0,11.000000
mean,1.0,3201.747747
std,0.0,22.415727
min,1.0,3161.124842
2.5%,1.0,3166.808529
50%,1.0,3198.977615
97.5%,1.0,3241.316913
max,1.0,3247.959665


In [10]:
birth_numerator = births.loc[(births.measure=='low_weight_births')
                            &(births.scenario=='baseline')].groupby(['input_draw']).sum()
birth_denominator = births.loc[(births.measure=='total_births')
                            &(births.scenario=='baseline')].groupby(['input_draw']).sum()

(birth_numerator / birth_denominator).describe(percentiles=[0.025,0.975])

,year,value
count,11.0,11.000000
mean,1.0,0.108053
std,0.0,0.003877
min,1.0,0.101257
2.5%,1.0,0.101655
50%,1.0,0.108863
97.5%,1.0,0.113164
max,1.0,0.113455


In [7]:
births.measure.unique()

array(['birth_weight_sum', 'low_weight_births', 'total_births'],
      dtype=object)

# wasting prevalence

In [13]:
wpt = pd.read_csv(output_dir + 'wasting_state_person_time.csv')
wpt = wpt.loc[(wpt.age.isin(['6-11_months','12_to_23_months','2_to_4']))
             &(wpt.scenario=='baseline')]
wpt.head()

,Unnamed: 0,sex,year,cause,measure,input_draw,scenario,value,diarrhea,sq_lns,mam_treatment,sam_treatment,age
48,48,female,2022,mild_child_wasting,state_person_time,29,baseline,0.0,cat2,covered,covered,covered,6-11_months
49,49,female,2022,mild_child_wasting,state_person_time,29,baseline,0.0,cat1,covered,covered,covered,6-11_months
50,50,female,2022,mild_child_wasting,state_person_time,29,baseline,0.0,cat2,covered,uncovered,covered,6-11_months
51,51,female,2022,mild_child_wasting,state_person_time,29,baseline,0.0,cat1,covered,uncovered,covered,6-11_months
52,52,female,2022,mild_child_wasting,state_person_time,29,baseline,0.0,cat2,covered,covered,uncovered,6-11_months


In [16]:
wpt_tot = wpt.groupby(['scenario','input_draw']).sum()
wpt_mam = wpt.loc[wpt.cause=='moderate_acute_malnutrition'].groupby(['scenario','input_draw']).sum()
wpt_sam = wpt.loc[wpt.cause=='severe_acute_malnutrition'].groupby(['scenario','input_draw']).sum()
wpt_gam = wpt.loc[wpt.cause.isin(['moderate_acute_malnutrition',
                                 'severe_acute_malnutrition'])].groupby(['scenario','input_draw']).sum()
prev_mam = (wpt_mam/wpt_tot).groupby('scenario').describe(percentiles=[0.025,0.975])[['value']]
prev_sam = (wpt_sam/wpt_tot).groupby('scenario').describe(percentiles=[0.025,0.975])[['value']]
prev_gam = (wpt_gam/wpt_tot).groupby('scenario').describe(percentiles=[0.025,0.975])[['value']]

prev_mam

value                                                              \
         count      mean       std       min      2.5%       50%     97.5%   
scenario                                                                     
baseline  11.0  0.074187  0.001006  0.072451  0.072547  0.074251  0.075566   

                    
               max  
scenario            
baseline  0.075605

In [17]:
prev_sam

value                                                             \
         count      mean       std       min     2.5%       50%     97.5%   
scenario                                                                    
baseline  11.0  0.017221  0.000611  0.016396  0.01643  0.017136  0.018371   

                   
              max  
scenario           
baseline  0.01853

In [18]:
prev_gam

value                                                            \
         count      mean       std       min     2.5%       50%    97.5%   
scenario                                                                   
baseline  11.0  0.091408  0.001538  0.089367  0.08937  0.091315  0.09386   

                    
               max  
scenario            
baseline  0.093981

# stunting

In [23]:
spt = pd.read_csv(output_dir + 'stunting_state_person_time.csv')
spt = spt.loc[(spt.age.isin(['6-11_months','12_to_23_months','2_to_4']))
             &(spt.scenario=='baseline')]
spt_tot = spt.groupby(['scenario','input_draw']).sum()
spt_mod = spt.loc[spt.cause=='cat2'].groupby(['scenario','input_draw']).sum()
spt_sev = spt.loc[spt.cause=='cat1'].groupby(['scenario','input_draw']).sum()
spt_all = spt.loc[spt.cause.isin(['cat2',
                                 'cat1'])].groupby(['scenario','input_draw']).sum()
prev_mod = (spt_mod/spt_tot).groupby('scenario').describe(percentiles=[0.025,0.975])[['value']]
prev_sev = (spt_sev/spt_tot).groupby('scenario').describe(percentiles=[0.025,0.975])[['value']]
prev_all = (spt_all/spt_tot).groupby('scenario').describe(percentiles=[0.025,0.975])[['value']]

In [24]:
prev_mod

value                                                              \
         count      mean       std       min      2.5%       50%     97.5%   
scenario                                                                     
baseline  11.0  0.187834  0.004989  0.181216  0.181242  0.187242  0.195054   

                    
               max  
scenario            
baseline  0.195737

In [25]:
prev_sev

value                                                              \
         count      mean       std       min      2.5%       50%     97.5%   
scenario                                                                     
baseline  11.0  0.174501  0.006374  0.161437  0.163138  0.174806  0.182075   

                    
               max  
scenario            
baseline  0.182216

In [27]:
prev_all

value                                                            \
         count      mean      std       min     2.5%       50%     97.5%   
scenario                                                                   
baseline  11.0  0.362335  0.00482  0.353525  0.35446  0.362047  0.368826   

                    
               max  
scenario            
baseline  0.368947

# dalys

In [46]:
ylls = pd.read_csv(output_dir + 'ylls.csv')
ylls['cause'] = np.where(ylls.cause.str.contains('malnutrition'),'pem',ylls.cause)
ylds = pd.read_csv(output_dir + 'ylds.csv')
ylds['cause'] = np.where(ylds.cause.str.contains('malnutrition'),'pem',ylds.cause)
ylls = (ylls.loc[(ylls.scenario=='baseline')
               &(ylls.age.isin(['6-11_months','12_to_23_months','2_to_4']))]
        .groupby(['input_draw','scenario','cause']).sum())[['value']]
ylds = (ylds.loc[(ylds.scenario=='baseline')
               &(ylds.age.isin(['6-11_months','12_to_23_months','2_to_4']))]
        .groupby(['input_draw','scenario','cause']).sum())[['value']]
dalys = (ylls + ylds).dropna()
dalys = dalys / wpt_tot[['value']].reset_index().set_index(['input_draw','scenario']) * 100_000
dalys.groupby(['scenario','cause']).describe(percentiles=[0.025,0.975])

value                             \
                                      count          mean          std   
scenario cause                                                           
baseline diarrheal_diseases            11.0  11188.806169  3015.168712   
         lower_respiratory_infections  11.0   5817.478847  1277.852817   
         measles                       11.0   1397.994257   845.019762   
         pem                           11.0   1995.916587   399.431614   

                                                                               \
                                               min         2.5%           50%   
scenario cause                                                                  
baseline diarrheal_diseases            6683.917071  6915.183073  11492.409409   
         lower_respiratory_infections  4296.571101  4336.878915   5843.033589   
         measles                        410.630999   428.908043   1184.500564   
         pem                           1448.129242  1482.922995   2048.901833   

                                                                   
                                              97.5%           max  
scenario cause                                                     
baseline diarrheal_diseases            15048.620943  15079.390291  
         lower_respiratory_infections   8022.501593   8241.753450  
         measles                        2874.576588   3097.820744  
         pem                            2645.822644   2741.273980

In [47]:
ylls = pd.read_csv(output_dir + 'ylls.csv')
ylls['cause'] = np.where(ylls.cause.str.contains('malnutrition'),'pem',ylls.cause)
ylds = pd.read_csv(output_dir + 'ylds.csv')
ylds['cause'] = np.where(ylds.cause.str.contains('malnutrition'),'pem',ylds.cause)
ylls = (ylls.loc[(ylls.scenario=='baseline')
               &(ylls.age.isin(['6-11_months','12_to_23_months','2_to_4']))]
        .groupby(['input_draw','scenario']).sum())[['value']]
ylds = (ylds.loc[(ylds.scenario=='baseline')
               &(ylds.age.isin(['6-11_months','12_to_23_months','2_to_4']))]
        .groupby(['input_draw','scenario']).sum())[['value']]
dalys = (ylls + ylds).dropna()
dalys = dalys / wpt_tot[['value']].reset_index().set_index(['input_draw','scenario']) * 100_000
dalys.groupby(['scenario']).describe(percentiles=[0.025,0.975])

value                                                         \
         count          mean          std           min          2.5%   
scenario                                                                
baseline  11.0  44019.200633  6362.792045  31422.369366  32216.934506   

                                                   
                   50%        97.5%           max  
scenario                                           
baseline  46681.597817  50584.87418  50893.602107

# dalys by wasting

In [49]:
ylls = pd.read_csv(output_dir + 'ylls.csv')
ylls.head()

,Unnamed: 0,sex,year,cause,measure,input_draw,scenario,value,wasting_state,age
0,0,female,2022,diarrheal_diseases,ylls,29,baseline,88.863181,susceptible_to_child_wasting,early_neonatal
1,1,female,2022,diarrheal_diseases,ylls,29,baseline,0.000000,mild_child_wasting,early_neonatal
2,2,female,2022,diarrheal_diseases,ylls,29,baseline,0.000000,moderate_acute_malnutrition,early_neonatal
3,3,female,2022,diarrheal_diseases,ylls,29,baseline,88.863181,severe_acute_malnutrition,early_neonatal
4,4,female,2022,diarrheal_diseases,ylls,29,baseline,266.484978,susceptible_to_child_wasting,late_neonatal
